#### 一. 隐式转换函数
1. 隐式转换函数:  
   1. 指用`implicit`声明的函数, 且该函数只有一个参数  
   2. 隐式转换函数把变量从一种转换为另一种类型, 并在需要的地方自动执行(后面介绍什么是需要的地方)

In [1]:
import java.io.File
implicit def string2File(src:String):File = new File(src)

import java.io.File

defined function string2File

2. 什么是需要的地方  
  1. 当参数类型和所传变量类型不一致时  
  2. 当对象访问不存在的成员变量/成员函数时  
  3. 当表达式类型与预期类型不一致时

#### 二. 隐式参数
1. 隐式参数  
  1. 指被`implicit`修饰的函数的参数  
  2. 隐士参数不用在调用方法时显式传入, 编译器会自动查找该类型的变量
  
2. 转换函数作为隐式参数  
 如下例子, 编译器自动查找隐式变量, 这个变量是一个函数, 可以把T类型转换为Ordered[T]

In [3]:
def smaller[T](a:T,b:T)(implicit order:T => Ordered[T]) = {
    if(a<b) a else b
}
print(smaller(40,2))

2

defined function smaller

#### 三. @implicitNotFound注解
1. 该注解告诉编译器, 在不能构造出带有该注解类型的参数时, 给出错误提示  
2. 如在类`<:<`的定义  
```scala
  @implicitNotFound(msg = "Cannot prove that ${From} <:< ${To}.")
  sealed abstract class <:<[-From, +To] extends (From => To) with Serializable
```

#### 四. canBuildFrom
1. `canbuildFrom`方法是广泛存在于所有scala集合类的伴生对象中的隐式转换函数, 在使用集合类时, 自动执行对应伴生对象中的隐式转换  
2. `canbuildFrom`隐式转换函数产生`CanBuildFrom`对象  
 以下使用Array类举例
```scala
// Array伴生对象中的canBuildFrom方法
implicit def canBuildFrom[T](implicit t: ClassTag[T]): CanBuildFrom[Array[_], T, Array[T]] =
  new CanBuildFrom[Array[_], T, Array[T]] {  // CanBuildFrom是trait,有2个抽象方法apply
    // apply方法返回Builder类的对象, 每种数据结构都有各自的Builder伴生对象和class, Array对应的ArrayBuilder
    def apply(from: Array[_]) = ArrayBuilder.make[T]()(t)   // 返回ArrayBuilder class
    def apply() = ArrayBuilder.make[T]()(t)
}
```
```scala
// 定义在TraversableLike的map方法
def repr: Repr = this.asInstanceOf[Repr]
def map[B, That](f: A => B)(implicit bf: CanBuildFrom[Repr, B, That]): That = {
    // repr是表现类型, 即数据结构类型, 比如Array. bf在Array中是由canBuildFrom方法产生的CanBuildFrom伴生对象
    // bf(repr)即为new CanBuildFrom.apply(Array), 因此b为ArrayBuilder类型
      val b = bf(repr) 
      for (x <- self)
        if (p(x)) b += f(x) // 将计算结果追加到ArrayBuilder对象b中
      b.result  // 返回结果
    }
```